# Detecting the fiducial particles in EM images

Fiducial particles/markers are used in correlated light and electron microscopy (CLEM) to enable accurate overlaying of fluorescence (LM) and electron microscopy (EM) images. The fiducial particles in EM images appear as bright circular regions with dark central spot. 

In this notebook, we **detect fiducial particles** in **EM images** using the template matching algorithm. As a template we use an artificially generated bright image with dark spot in the middle that resembles with its appearence the central part of the fiducial particle. After matching this template to the individual fiducial particles we detect fiducial clusters that consists of at least three fiducial particles in close proximity of each other. The main steps of this algorithm are:
- **1. Fiducial particle detection** - Detection of fiducial particles using the template matching algorithm.
- **2. Cluster detection** - Filtering the set of individual fiducial particles by recognizing clusters of touching fiducial particles and replacing these clusters by their centroid positions.
- **3. Results saving** - Saving the positions of all detected fiducial particles and the positions of fiducial clusters into files.

Load the necessary python libraries:

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import cv2
import math
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt
from imutils.object_detection import non_max_suppression
from utils import plot_image, list_to_dataframe, dataframe_to_nparray, dataframe_to_xml, dataframe_to_pointcloud, dataframe_to_xml_

Set the path to the input EM image and values for the parameters, load the EM image:

In [ ]:
from pathlib import Path

# Load the imput EM image and template
input_folder = 'E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos2'
image_path = Path(os.path.join(input_folder, "240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos2_bin2_EM.tif"))

#image_path = Path('E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos1/240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_bin4_EM_small.tif')
#image_path = Path('E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos2/240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos2_bin4_EM.tif')
#image_path = Path('E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos3/240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos3_bin4_EM.tif')
print(image_path.exists())

output_folder = Path(os.path.join(input_folder,"output"))
output_folder.mkdir(exist_ok=True)

test_folder = Path('//vironova.com/root/Users/kristinal/Documents/1Test')

#template_path = Path('E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos1/240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_bin4_EM_template.tif')
#print(template_path.exists())

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)

# Parameters for template generation    #Sample1/bin 2 needs size 11, otherwise size 9 works fine
template_size = 9     # Create an empty template of size ('size','size') filled with constant value - 'value',9,11
template_value = 80    #80,90

# Parameters for template matching
matching_threshold = 0.7  # threshold for template matching
overlap_threshold =0.1

# Get template dimensions
#h, w = template.shape

# Information about fiducial particles
fiducial_diam = 27     # diameter of fiducial particles in px

In [ ]:
from skimage import exposure
def normalize_image(image):
    min_val = np.min(image)
    max_val = np.max(image)
    return ((image - min_val) / (max_val - min_val) * 255).astype(np.uint8)

def standardize_image(image):
    mean = np.mean(image)
    std = np.std(image)
    corrected = (image - mean) / std
    corrected = exposure.rescale_intensity(corrected, in_range='image', out_range=(0, 1))
    corrected = np.uint8(corrected*255)
    return corrected

standardized_image = standardize_image(image)
normalized_image = normalize_image(image)


Perform the illumination correction on the image to make the algorithm more robust:

In [ ]:
from skimage.transform import rescale, resize
from skimage import io, filters, color, exposure

def illumination_correction(image, sigma=80):
    image_rescaled = rescale(image, 0.25, anti_aliasing=True)
    background = filters.gaussian(image_rescaled, sigma=sigma)
    background = resize(background, image.shape, anti_aliasing=True)
    corrected = image - background
    corrected = filters.gaussian(corrected, sigma=1)         # smooth the image a little bit to make nicer segmentation later
    #corrected = filters.median(corrected)         # smooth the image a little bit to make nicer segmentation later
    corrected = exposure.rescale_intensity(corrected, in_range='image', out_range=(0, 1))
    corrected = np.uint8(corrected*255)
    return background, corrected

background, corrected_image = illumination_correction(image, 80)

fig = plt.figure(figsize=(15,20))
ax1 = fig.add_subplot(1,3,1)
ax1.imshow(image,cmap="gray")
ax1.set_title('Original image')
plt.axis('off')

ax2 = fig.add_subplot(1,3,3)
ax2.imshow(normalized_image,cmap="gray")
ax2.set_title('Corrected image')
plt.axis('off')

ax1 = fig.add_subplot(1,3,2)
ax1.imshow(background,cmap="gray")
ax1.set_title('Background')
plt.axis('off')

In [ ]:
print(np.mean(image))
print(np.mean(normalized_image))
print(np.mean(standardized_image))
print(np.mean(corrected_image))

## 1. Fiducial particle detection

### Creating a template for the template matching

Fiducial particles have a very distinct central region, characterized by a dark spot surrounded by a lighter gray region. Instead of attempting to match the entire particle using a full template, we focus on detecting the central region only. To accomplish this, the template can be easily generated artificially. We create an initial example template of size (9,9). This example template will be immediately resized to the desired template size which was set up earlier using parameter 'template_size'.

Using a full particle template did not work satisfactory because of the variaty of the region around the particle. 

In [ ]:
# Create the template as an empty template of size ('size','size') filled with constant value - 'value'
example_template = np.full((9, 9), template_value, dtype=np.uint8)

# Define the 3x3 pattern
template_pattern = np.array([[4, 2, 4],
                             [2, 0, 2],
                             [4, 2, 4]])

# Calculate the starting index to place the pattern in the middle of the template
s_idx = (example_template.shape[0] - template_pattern.shape[0]) // 2
e_idx = s_idx + template_pattern.shape[0]
    
# Place the pattern in the middle of the template
example_template[s_idx:e_idx, s_idx:e_idx] = template_pattern

#import bigfish.stack as stack
#template = stack.resize_image(template,(template.shape[0]*2,template.shape[0]*2), method='bilinear')

example_template = example_template.astype(np.uint8)
template = cv2.resize(example_template, (template_size,template_size))

In [ ]:
plot_image(template,(10))
print(template)

Helping functions for template matching. 

In [ ]:
def non_max_suppression(boxes, scores, threshold):
    # Sort boxes by score in descending order
    sorted_indices = np.argsort(scores)[::-1]
    # print("Indices: ", sorted_indices)
    
    keep_boxes = []
    
    while sorted_indices.size > 0:
        # Pick the box with the highest score
        box_id = sorted_indices[0]
        keep_boxes.append(box_id)
        
        # Calculate IoU of the picked box with the rest
        ious = calculate_iou(boxes[box_id], boxes[sorted_indices[1:]])
        
        # Remove boxes with IoU over the threshold
        keep_indices = np.where(ious < threshold)[0]
        
        # Update the indices
        sorted_indices = sorted_indices[keep_indices + 1]
    print("Keep boxes: ", keep_boxes)
    return keep_boxes

def calculate_iou(box, boxes):
    # Calculate intersection areas
    x1 = np.maximum(box[0], boxes[:, 0])
    y1 = np.maximum(box[1], boxes[:, 1])
    x2 = np.minimum(box[2], boxes[:, 2])
    y2 = np.minimum(box[3], boxes[:, 3])
    
    intersection_area = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)
    
    # Calculate union areas
    box_area = (box[2] - box[0]) * (box[3] - box[1])
    boxes_area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
    union_area = box_area + boxes_area - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area
    return iou

def template_matching(image, template, threshold):
    result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    #plot_image(result)
    locations = np.where(result >= threshold)
    scores = result[locations]
    matches = list(zip(*locations[::-1]))

    return matches, scores


Template matching is performed. The matches are first filtered so we ensure that the matches are unique and not overlapping. Then we filter false positives by looking at the 

In [ ]:
# Perform template matching
matching_threshold = 0.7 #0.68
image2 = normalized_image #image #
#image2 = standardized_image
#image2 = corrected_image
matches, scores = template_matching(image2, template, matching_threshold)
print('Matches:', len(matches), matches)

# Create bounding boxes
w, h = template.shape[::-1]
boxes = [(x, y, x + w, y + h) for (x, y) in matches]    # x,y is the probably the middle of the box, but it does not matter when calculating 
                                                        # the box overlap
# Apply non-maximum suppression for filtering out overlapping boxes
keep_ids = non_max_suppression(np.array(boxes), scores, overlap_threshold)
print('Keep ids:',len(keep_ids), keep_ids)

# Center the matches
loc = [np.asarray(matches[keep_id])+[w//2,h//2] for keep_id in keep_ids]


In [ ]:
def otsu_threshold_1d(data, nbins=100):
    # Compute histogram and bin edges
    hist, bin_edges = np.histogram(data, bins=nbins)
    bin_mids = (bin_edges[:-1] + bin_edges[1:]) / 2

    # Normalize histogram to get probabilities
    hist = hist.astype(float)
    total = hist.sum()
    prob = hist / total

    # Cumulative sums and means
    cumulative_prob = np.cumsum(prob)
    cumulative_mean = np.cumsum(prob * bin_mids)
    global_mean = cumulative_mean[-1]

    # Compute between-class variance for all thresholds
    numerator = (global_mean * cumulative_prob - cumulative_mean) ** 2
    denominator = cumulative_prob * (1 - cumulative_prob)
    # Avoid division by zero
    denominator[denominator == 0] = 1e-10
    sigma_b_squared = numerator / denominator

    # Find the threshold with the maximum between-class variance
    idx = np.argmax(sigma_b_squared)
    threshold = bin_mids[idx]
    return threshold

def average_perimeter_intensity(image, center, radius):
    # Create a circular mask
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.circle(mask, center, radius, 255, 1)
    
    # Extract perimeter pixels
    perimeter_pixels = image[mask == 255]
    
    # Calculate average intensity
    average_intensity = np.mean(perimeter_pixels)
    #print("Average perimeter intensity: ", average_intensity)
    
    return average_intensity

def average_intensity_square(image, middle, size):
    x, y = middle
    square = image[y-size:y+size+1, x-size:x+size+1].copy()
    return np.mean(square)

def average_intensity_square_ring(image, middle, out_size, in_size):
    x, y = middle
    square = image[y-out_size:y+out_size+1, x-out_size:x+out_size+1].copy()
    square[in_size:-in_size,in_size:-in_size]=0
    return np.average(square[square!=0])

def filter_the_template_matching_results(locs, image, size, out_size, in_size):
    intensities_middle = []
    intensities_square_ring = []
    for loc in locs:
        intensities_middle.append(average_intensity_square(image, loc, size)) 
        intensities_square_ring.append(average_intensity_square_ring(image, loc, out_size, in_size))

    thresh_middle = otsu_threshold_1d(intensities_middle, 70)
    thresh_square_ring = otsu_threshold_1d(intensities_square_ring, 70)

    fig, axes = plt.subplots(1, 2, figsize=(15, 10), layout='constrained')
    axes[0].hist(intensities_middle, bins=100, color='blue', edgecolor='black')
    axes[0].set_title('Histogram of Intensities Middle')
    axes[0].set_xlabel('Intensity')
    axes[0].set_ylabel('Frequency')

    # Plot histogram for intensities_square_ring on the second subplot
    axes[1].hist(intensities_square_ring, bins=100, color='green', edgecolor='black')
    axes[1].set_title('Histogram of Intensities Square Ring')
    axes[1].set_xlabel('Intensity')
    axes[1].set_ylabel('Frequency')

    print("Threshold middle: ", thresh_middle)
    print("Threshold square ring: ", thresh_square_ring)

    locs1 = [loc for loc in locs if average_intensity_square(image, loc, size) < thresh_middle]                # test 1
    locs2 = [loc for loc in locs if average_intensity_square_ring(image, loc, out_size, in_size) < thresh_square_ring]   # test 2
    locs3 = [loc for loc in locs1 if average_intensity_square_ring(image, loc, out_size, in_size) < thresh_square_ring]  # only if passed both tests
    return locs1, locs2, locs3

# Filter the template matching results based on average intensity
loc1, loc2, loc3 = filter_the_template_matching_results(loc,  image2, 1, 6, 3) # size of middle square, size of outer square, size of inner square for ring

# Draw rectangles around the matched regions
img2 = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
for pt in loc1:
    cv2.circle(img2, (pt[0] + w , pt[1] + h ), 1, (0, 128, 0), 2)
      
for pt in loc2:
    cv2.circle(img2, (pt[0] - w , pt[1] - h ), 1, (255, 255, 0), 2)

for pt in loc3:
    cv2.rectangle(img2, (pt[0] - fiducial_diam//2 , pt[1] - fiducial_diam//2 ), (pt[0] + fiducial_diam//2 , pt[1] + fiducial_diam//2), (0, 0, 0), 2)
    
cv2.imwrite(str(output_folder/'fiducial_detectionttt.png'), img2)

In [ ]:

'''
radius = round(template_size/2)+2  # +1 or +2 
#loc = [np.asarray(matches[keep_id])+[w//2,h//2] for keep_id in keep_ids if average_perimeter_intensity(image2, np.asarray(matches[keep_id])+[w//2,h//2], radius) < 110]
#loc2 = [np.asarray(matches[keep_id])+[w//2,h//2] for keep_id in keep_ids if average_perimeter_intensity(image2, np.asarray(matches[keep_id])+[w//2,h//2], radius) >= 110]
#loc = [np.asarray(matches[keep_id])+[w//2,h//2] for keep_id in keep_ids if average_intensity_square(image, np.asarray(matches[keep_id])+[(w//2)-1,(h//2)-1], 3) < 95] #80,100
loc = [np.asarray(matches[keep_id])+[w//2,h//2] for keep_id in keep_ids if average_intensity_square_ring(image2, np.asarray(matches[keep_id])+[w//2,h//2], 6, 3) < 135] #80,100
#loc = [np.asarray(matches[keep_id])+[w//2,h//2] for keep_id in keep_ids if average_intensity_square(image, np.asarray(matches[keep_id])+[(w//2)-fiducial_diam//2,(h//2)-fiducial_diam//2], 
#     
#                                                                                                                                       fiducial_diam) < 130] #110

#loc2 = [np.asarray(matches[keep_id])+[w//2,h//2] for keep_id in keep_ids if average_intensity_square(image, np.asarray(matches[keep_id])+[(w//2)-fiducial_diam//2,(h//2)-fiducial_diam//2], 
#                                                                                                                                          fiducial_diam) < 111] #110, 130

#loc4 = [np.asarray(matches[keep_id])+[w//2,h//2] for keep_id in keep_ids if average_intensity_square(image, np.asarray(matches[keep_id])+[(w//2)-1,(h//2)-1], 3) >= 100] #80



# Draw rectangles around the matched regions
img2 = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#for pt in loc:
    #cv2.rectangle(img2, (pt[0] - fiducial_diam//2 , pt[1] - fiducial_diam//2 ), (pt[0] + fiducial_diam//2 , pt[1] + fiducial_diam//2), (0, 255, 0), 2)
    #cv2.rectangle(img2, (pt[0] - fiducial_diam//2 , pt[1] - fiducial_diam//2 ), (pt[0] + fiducial_diam//2 , pt[1] + fiducial_diam//2), (0, 255, 0), 2)
    # plot points in the middle of the box
#     cv2.circle(img2, (pt[0] - w , pt[1] - h ), 1, (0, 255, 0), 2)

for pt in loc:
    cv2.rectangle(img2, (pt[0] - fiducial_diam//2 , pt[1] - fiducial_diam//2 ), (pt[0] + fiducial_diam//2 , pt[1] + fiducial_diam//2), (255, 255, 0), 2)
    
    # plot points in the middle of the box
    #cv2.circle(img2, (pt[0] - w , pt[1] - h ), 1, (255, 255, 0), 2)

for pt in loc3:
    #cv2.rectangle(img2, (pt[0] - fiducial_diam//2 , pt[1] - fiducial_diam//2 ), (pt[0] + fiducial_diam//2 , pt[1] + fiducial_diam//2), (255, 255, 0), 2)
    
    # plot points in the middle of the box
    cv2.circle(img2, (pt[0] - w , pt[1] - h ), 1, (0, 255, 0), 2)
for pt in loc4:
    #cv2.rectangle(img2, (pt[0] - fiducial_diam//2 , pt[1] - fiducial_diam//2 ), (pt[0] + fiducial_diam//2 , pt[1] + fiducial_diam//2), (255, 255, 0), 2)
    
    # plot points in the middle of the box
    cv2.circle(img2, (pt[0] - w , pt[1] - h ), 1, (255, 255, 0), 2)
'''
cv2.imwrite(str(output_folder/'fiducial_detection8.png'), img2)

In [ ]:
avrg_intensity = []
for keep_id in keep_ids:
     avrg_intensity.append(average_intensity_square_ring(image2, np.asarray(matches[keep_id]), 5, 3))

print(avrg_intensity)

plt.hist(avrg_intensity, bins=100)  # 'bins' can be adjusted as needed
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of Numbers')
plt.show()


In [ ]:
from scipy.signal import find_peaks
avrg_intensity = [average_intensity_square(image, np.asarray(matches[keep_id])+[(w//2)-radius,(h//2)-radius], radius) for keep_id in keep_ids]
#avrg_intensity =  [average_intensity_square(image, np.asarray(matches[keep_id])+[(w//2)-2,(h//2)-2], 2+2) for keep_id in keep_ids]
find_peaks(avrg_intensity, 100)
plt.figure(figsize=(10, 6))
plt.hist(avrg_intensity, bins=30, edgecolor='black')

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of Values')

# Display the plot
plt.show()

In [ ]:
from matplotlib import pyplot as plt
plot_image(img2)
#plt.savefig(str(output_folder/'fiducial_detection.png'))


### Filtering of the detected fiducial particles 

Clusters of fiducial particles (FP) will be replaced by 1 point located in the centre of the cluster. Single FP will be removed.

In [ ]:
# Create a black image of the same size as img2, then draw filled circles with the radius of FP at the locations 
# of FP taken from the loc variable, dilate the binary image by circular kernel to connect nearby FP, save the image

img_mask = np.zeros_like(img2)
for pt in loc:
    # draw a filled circle around the fiducial particle
    cv2.circle(img_mask, pt, fiducial_diam//2, (255, 255, 255), -1)
    # draw a filled rectangle around the fiducial particle
    #cv2.rectangle(img_mask, pt, (pt[0] + w, pt[1] + h), (255, 255, 255), -1)

# Dilate img_mask by circluar kernel of size 7x7
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
img_mask = cv2.dilate(img_mask, kernel, iterations=1)

plot_image(img_mask)
plt.savefig(str(output_folder/'fiducial_detection_mask.png'))


In [ ]:
# Calculate for each connected component the centroid and the pixel area, if the area is larger then 
# the area of 3 fiducial particles then keep the FP, save the centroid and the area into a list

# Find connected components
_, labels = cv2.connectedComponents(img_mask)

# Calculate the centroid and area of each connected component
centroids = []
areas = []

for label in np.unique(labels):
    if label == 0:             # skip the background
        continue
    
    mask = np.zeros_like(img_mask, dtype=np.uint8)
    mask[labels == label] = 1
    
    moments = cv2.moments(mask)
    #print("Moments: ", moments)

    if moments["m00"] > (3 * math.pi * (fiducial_diam//2)**2) :   # equivalent to the area of 3 fiducial particles
        centroids.append((int(moments["m10"] / moments["m00"]), int(moments["m01"] / moments["m00"])))
        areas.append(int(moments["m00"]))
        #print(moment["m00"])

print("Centroids: ", centroids)
print("Areas: ", areas)

# Draw the centroids on the original image
img3 = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
for centroid in centroids:
    cv2.circle(img3, centroid, 5, (0, 255, 0), -1)

plot_image(img3)

cv2.imwrite(str(output_folder/'fiducial_detection_centroids.png'), img3)


#### Save the FP locations and filtered FP locations

Save the FP locations and filtered FP locations into a pandas dataframe with columns: 'id', 'name', 'pos_x', 'pos_y'. Then convert dataframe into numpy array of coordinate pairs (X,Y), XML file and PLY file

In [ ]:
loc

In [ ]:
# Save all detected fiducial particles into a Pandas dataframe

target_all_df = list_to_dataframe(loc) #str(output_folder/"target_all_df.csv")

# target = dataframe_to_nparray(target_all_df)
dataframe_to_xml_(target_all_df,str(output_folder/"target_all.xml"))                      # str(output_folder/"target.xml")
target_pcd = dataframe_to_pointcloud(target_all_df, str(output_folder/"target_all.ply"))  # "str(output_folder/target.ply)"

## -----------------------------------------------------------------------------------------------

target_filt_df = list_to_dataframe(centroids) #str(output_folder/"target_filt_df.csv")

#target_filt = dataframe_to_nparray(target_filt_df)
dataframe_to_xml_(target_filt_df, str(output_folder/"target_filtered.xml"))
target_filt_pcd = dataframe_to_pointcloud(target_filt_df, str(output_folder/"target_filtered.ply"))  # "target.ply"

